### 5.4

#### 5.4.1

In [0]:
import numpy as np

In [0]:
class MulLayer:
  def __init__(self):
    self.x = None
    self.y = None
  
  def forward(self, x, y):
    self.x = x
    self.y = y
    out = x * y
    return out
 
  def backward(self, dout):
    dx = dout * self.y
    dy = dout * self.x
    
    return dx, dy

In [0]:
apple = 100
apple_num = 2
tax = 1.1

In [0]:
#layer
mul_apple_layer = MulLayer()
mul_tax_layer = MulLayer()

In [0]:
#forward
apple_price = mul_apple_layer.forward(apple, apple_num)
price = mul_tax_layer.forward(apple_price, tax)

In [6]:
price

220.00000000000003

In [0]:
#backward
dprice = 1
dapple_price, dtax = mul_tax_layer.backward(dprice)
dapple, dapple_num = mul_apple_layer.backward(dapple_price)

In [0]:
print(dapple, dapple_num, dtax)

#### 5.4.2

In [0]:
class AddLayer:
  def __init__(self):
    pass
  def forward(self, x, y):
    out = x + y
    return out
  def backward(self, dout):
    dx = dout * 1
    dy = dout * 1
    return dx, dy

In [0]:
apple = 100
apple_num = 2
orange = 150
orange_num = 3
tax = 1.1

In [0]:
#layer
mul_apple_layer = MulLayer()
mul_orange_layer = MulLayer()
add_apple_orange_layer = AddLayer()
mul_tax_layer = MulLayer()

In [0]:
# forward
apple_price = mul_apple_layer.forward(apple, apple_num)
orange_price = mul_orange_layer.forward(orange, orange_num)
all_price = add_apple_orange_layer.forward(apple_price, orange_price)
price = mul_tax_layer.forward(all_price, tax)

In [0]:
#backward
dprice = 1
dall_price,dtax = mul_tax_layer.backward(dprice)
dapple_price,dorange_price = add_apple_orange_layer.backward(dall_price)
dorange, dorange_num = mul_orange_layer.backward(dorange_price)
dapple, dapple_num = mul_apple_layer.backward(dapple_price)

In [0]:
print(price)

715.0000000000001


In [0]:
print(dapple_num, dapple, dorange,dorange_num, dtax)

110.00000000000001 2.2 3.3000000000000003 165.0 650


### 5.5

#### 5.5.1

In [0]:
class ReLu:
  def __init__(self):
    self.mask = None
  
  def forward(self, x):
    self.mask = (x <= 0)
    out = x.copy()
    out[self.mask] = 0
    
    return x
  
  def backward(self, dout):
    dout[self.mask] = 0
    dx = dout
    
    return dx

#### 5.5.2

In [0]:
class Sigmoid:
  def __init__(self):
    self.out = None
  def forward(self,x):
    out = 1 / (1 + np.exp(-x))
    self.out = out
    
    return out
  def backward(self, dout):
    dx = dout * (1.0 - self.out) * self.out
    return dx

### 5.6

#### 5.6.2

In [0]:
class Affine:
  def __init__(self,W, b):
    self.W = W
    self.b = b
    self.x = None
    self.dW = None
    self.db =None
  
  def forward(self, x):
    self.x = x
    out = np.dot(x, self.W) + self.b
    
    return out
  
  def backward(self,dout):
    dx = np.dot(dout,self.W.T)
    self.dW = np.dot(self.X.T, dout)
    self.db = np.sum(dout, axis=0)
    return dx

#### 5.6.3

In [0]:
class SotmaxWithLoss:
  def __init__(self):
    self.loss = None
    self.y = None
    self.t =None
  def forward(self, x, t):
    self.t = t
    self.y = softmax(x)
    self.loss = cross_entropy_error(self.y, self.t)
    
    return self.loss
  
  def backword(self, dout=1):
    batch_size = self.t.shape[0]
    dx = (self.y - self.t) / batch_size
    
    return dx

### 5.7

#### 5.7.2

In [0]:
import numpy as np
from collections import OrderedDict

In [0]:
def numerical_gradient(f, x):
    h = 1e-4 # 0.0001
    grad = np.zeros_like(x)
    
    it = np.nditer(x, flags=['multi_index'], op_flags=['readwrite'])
    while not it.finished:
        idx = it.multi_index
        tmp_val = x[idx]
        x[idx] = float(tmp_val) + h
        fxh1 = f(x) # f(x+h)
        
        x[idx] = tmp_val - h 
        fxh2 = f(x) # f(x-h)
        grad[idx] = (fxh1 - fxh2) / (2*h)
        
        x[idx] = tmp_val # 还原值
        it.iternext()   
        
    return grad

In [0]:
def cross_entropy_error(y, t):
    if y.ndim == 1:
        t = t.reshape(1, t.size)
        y = y.reshape(1, y.size)
        
    # 监督数据是one-hot-vector的情况下，转换为正确解标签的索引
    if t.size == y.size:
        t = t.argmax(axis=1)
             
    batch_size = y.shape[0]
    return -np.sum(np.log(y[np.arange(batch_size), t] + 1e-7)) / batch_size



In [0]:
class Affine:
    def __init__(self, W, b):
        self.W =W
        self.b = b
        
        self.x = None
        self.original_x_shape = None
        # 权重和偏置参数的导数
        self.dW = None
        self.db = None

    def forward(self, x):
        # 对应张量
        self.original_x_shape = x.shape
        x = x.reshape(x.shape[0], -1)
        self.x = x

        out = np.dot(self.x, self.W) + self.b

        return out

    def backward(self, dout):
        dx = np.dot(dout, self.W.T)
        self.dW = np.dot(self.x.T, dout)
        self.db = np.sum(dout, axis=0)
        
        dx = dx.reshape(*self.original_x_shape)  # 还原输入数据的形状（对应张量）
        return dx

In [0]:
class Relu:
  def __init__(self):
    self.mask = None
  def forward(self, x):
    self.mask = (x <= 0)
    out = x.copy()
    out[self.mask] = 0
    return out
  def backward(self, dout):
    dout[self.mask] = 0
    dx = dout
    return dx

In [0]:
class SoftmaxWithLoss:
    def __init__(self):
        self.loss = None
        self.y = None # softmax的输出
        self.t = None # 监督数据

    def forward(self, x, t):
        self.t = t
        self.y = softmax(x)
        self.loss = cross_entropy_error(self.y, self.t)
        
        return self.loss

    def backward(self, dout=1):
        batch_size = self.t.shape[0]
        if self.t.size == self.y.size: # 监督数据是one-hot-vector的情况
            dx = (self.y - self.t) / batch_size
        else:
            dx = self.y.copy()
            dx[np.arange(batch_size), self.t] -= 1
            dx = dx / batch_size
        
        return dx 

In [0]:
class TwoLayerNet:
  
  
  def __init__(self, input_size, hidden_size, output_size,
              weight_init_std=0.01):
    
    self.params= {}
    self.params['W1'] = weight_init_std * np.random.randn(input_size, hidden_size)
    self.params['b1'] = np.zeros(hidden_size)
    self.params['W2'] = weight_init_std * np.random.randn(hidden_size, output_size)
    self.params['b2'] = np.zeros(output_size)
    
    self.layers = OrderedDictr() # 有序字典按元素添加顺序
    self.layers['Affine1'] = Affine(self,params['W1'], self.params['b1'])
    self.layers['Relu1'] = Relu()
    
    self.layers['Affine2'] = Affine(self.params['W2'], self.params['b2'])
    self.lastLayer = SoftmaxWithLoss()
    
  def predict(self, x):
    for layer in self.layers.values():
      x = layer.forward(x)
      
    return x
  
  
  def loss(self, x, t):
    y = self.predict(x)
    return self.lastLayer.forward(y, t)
  
  def accuracy(self, x, t):
    y = self.predict(x)
    y = np.argmax(y, axis=1)
    if t.ndim!=1:
      t = np.argmax(t, axis=1)
    accuracy = np.sum(y == t) / float(x.shape[0])
    return accuracy
  
  def numerical_gradient(self, x, t):
    loss_W = lambda W:self.loss(x, t)
    grads = {}
    grads['W1'] = numerical_gradient(loss_W, self.params['W1'])
    grads['b1'] = numerical_gradient(loss_W, self.params['b1'])
    grads['W@'] = numerical_gradient(loss_W, self.params['W2'])
    grads['W2'] = numerical_gradient(loss_W, self.params['W2'])
    return grads
  
  def gradient(self, x, t):
    # forward
    self.loss(x, t)
    
    #backward
    dout = 1
    dout = self.lastLayer.backwarad(dout)
    
    layers = list(self.layers.values())
    layers.revers()
    
    for layer in layers:
      dout = layer.backward(dout)
    
    
    grads = {}
    grads['W1'] = self.layers['Affine1'].dW
    grads['b1'] = self.layers['Affine1'].db
    grads['W2'] = self.layers['Affine2'].dW
    grads['b2'] = self.layers['Affine2'].db
    
    return grads